## Create ground truth data
The goal of this notebook is to create a ground truth data to evaluate retrieval and RAG flow metrics.

In [18]:
from services import *
from common.settings import Settings
from common.client_factory import ClientFactory
import pandas as pd
from pandas import DataFrame
import json
import os
from typing import Any, Dict, List
from pprint import pprint
from tqdm import tqdm

### Read a dataset with documetns ids.

In [2]:
doc_with_ids_path = "/home/jovyan/work/notebook/retrieval_evaluation/dataset_with_doc_ids.csv"
ground_truth_path = "/home/jovyan/work/notebook/retrieval_evaluation/ground_truth.csv"

In [3]:
df = pd.read_csv(doc_with_ids_path, delimiter=";")
df[:1]

,source_system,section,question,answer,document_id
0,evdi,Analysekonzept,Wie läuft der Analyseprozess für Immobilienpro...,Der Analyseprozess bei Engel & Völkers Digital...,f2624f5125f9


In [4]:
settings = Settings()
settings.to_dict()

{'elastic_search_url': 'http://elasticsearch:9200',
 'index_name': 'documents',
 'postgres_user': 'admin',
 'postgres_password': 'P@ssw0rd!',
 'postgres_port': '5432',
 'postgres_db': 'rag_application',
 'source_system': 'evdi',
 'postgres_host': 'db',
 'embedding_model_name': 'distiluse-base-multilingual-cased-v1'}

In [5]:
factory = ClientFactory(settings)
es_client = factory.create_elasticsearch_client()
es_client.info()

ObjectApiResponse({'name': 'a639f0e3a42f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'aJrw85pRTECvuVEC8ISEHg', 'version': {'number': '8.9.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '8aa461beb06aa0417a231c345a1b8c38fb498a0d', 'build_date': '2023-07-19T14:43:58.555259655Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Create 5 questions to every question/answer pair
Create LLM client and request creating 5 questions for each question/answer pair.

In [6]:
bedrock_client = factory.create_bedrock_client()

In [7]:
def invoke_llm_model(prompt: str, bedrock_client)-> str:
    model = "amazon.titan-text-premier-v1:0"

    text_generation_config: dict[str, int | float | str] = {
        "maxTokenCount": 2000
    }

    body_request = json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": text_generation_config
        }
    )

    response = bedrock_client.invoke_model(
        modelId=model,
        contentType="application/json",
        accept="*/*",
        body=body_request,
    )

    body_as_plain_text = response.get("body").read()
    response_body = json.loads(body_as_plain_text)
    return response_body
    # return response_body["results"][0]["outputText"].strip()

In [8]:
def create_prompt_for_document(document: Dict[str, Any]) -> str:
    question = document["question"].strip()
    answer = document["answer"].strip()
    category = document["category"].strip()
    
    prompt_en = (
        "Act like you are an experienced customer support service specialist at EV Digital Invest AG company. \n"
        "You are tasked to write five new questions for the given content.  \n"
        "Follow the instructions below to prepare the data. \n"

        "# Instructions \n"
        "- Read a question, answer, and the related category name from the internal database. \n"
        "- Analyze the question, answer, and the related category. \n"
        "- Based on analytical outcome, create 5 new questions. \n"
        "- Format this questions as CSV file with the semicolon as a delimiter in the format: \n"
        "category;question \n"
        "{category};{question number 1} \n"
        "{category};{question number 2} \n"
        "{category};{question number 3} \n"
        "{category};{question number 4} \n"
        "{category};{question number 5} \n"

        "# Question, answer, and the related category from the database: \n"
        "Question: {Question} \n"
        "Answer: {Answer} \n"
        "Category: {Category} \n"
    )

    prompt = (
        "Verhalte Dich, als wärst Du ein erfahrener Kundendienstspezialist bei der EV Digital Invest AG. \n"
        "Deine Aufgabe ist es, fünf neue Fragen zum gegebenen Inhalt zu formulieren. \n"
        "Befolge die untenstehenden Anweisungen, um die Daten vorzubereiten. \n"

        "# Anweisungen \n"
        "- Lies eine Frage, Antwort und den dazugehörigen Abschnittsnamen aus der internen Datenbank. \n"
        "- Analysiere die Frage, Antwort und den zugehörigen Abschnitt. \n"
        "- Erstelle basierend auf dem Analyseergebnis 5 neue Fragen. \n"
        "- Formatiere diese Fragen als CSV-Datei mit Semikolon als Trennzeichen im folgenden Format: \n"
        "Abschnitt;Frage \n"
        "{Abschnitt};{Frage Nummer 1} \n"
        "{Abschnitt};{Frage Nummer 2} \n"
        "{Abschnitt};{Frage Nummer 3} \n"
        "{Abschnitt};{Frage Nummer 4} \n"
        "{Abschnitt};{Frage Nummer 5} \n"

        "# Frage, Antwort und der zugehörige Abschnitt aus der Datenbank: \n"
        f"Frage: {question} \n"
        f"Antwort: {answer} \n"
        f"Abschnitt: {category} \n"
    )
    
    return prompt

#### Generate ground truth questions for the first item to test the approach

In [9]:
first_doc = df.iloc[0]
first_doc.to_dict()

{'source_system': 'evdi',
 'section': 'Analysekonzept',
 'question': 'Wie läuft der Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest ab und welche externen Partner sind daran beteiligt?',
 'answer': 'Der Analyseprozess bei Engel & Völkers Digital Invest basiert auf einem speziell entwickelten Konzept für die Projektfinanzierung von Bauvorhaben und Grundstücken. Zunächst erfolgt eine interne Analyse, die auf dem umfangreichen Know-how des Engel & Völkers-Netzwerks aufbaut, einschließlich Research-Material und persönlicher Kontakte vor Ort. Um eine unabhängige Validierung zu gewährleisten, werden zusätzlich externe Experten hinzugezogen: Die bulwiengesa appraisal GmbH führt eine Markt- und Standortanalyse durch, CBRE übernimmt die technische Projektanalyse, Mazars analysiert den Projektentwickler, und HUTH DIETRICH HAHN prüft die rechtlichen Aspekte. Basierend auf diesen Gutachten erstellt Engel & Völkers Digital Invest eigenverantwortlich die finale Projekteinsch

In [10]:
prompt = create_prompt_for_document(first_doc.to_dict())
pprint(prompt)

('Verhalte Dich, als wärst Du ein erfahrener Kundendienstspezialist bei der EV '
 'Digital Invest AG. \n'
 'Deine Aufgabe ist es, fünf neue Fragen zum gegebenen Inhalt zu '
 'formulieren. \n'
 'Befolge die untenstehenden Anweisungen, um die Daten vorzubereiten. \n'
 '# Anweisungen \n'
 '- Lies eine Frage, Antwort und den dazugehörigen Abschnittsnamen aus der '
 'internen Datenbank. \n'
 '- Analysiere die Frage, Antwort und den zugehörigen Abschnitt. \n'
 '- Erstelle basierend auf dem Analyseergebnis 5 neue Fragen. \n'
 '- Formatiere diese Fragen als CSV-Datei mit Semikolon als Trennzeichen im '
 'folgenden Format: \n'
 'Abschnitt;Frage \n'
 '{Abschnitt};{Frage Nummer 1} \n'
 '{Abschnitt};{Frage Nummer 2} \n'
 '{Abschnitt};{Frage Nummer 3} \n'
 '{Abschnitt};{Frage Nummer 4} \n'
 '{Abschnitt};{Frage Nummer 5} \n'
 '# Frage, Antwort und der zugehörige Abschnitt aus der Datenbank: \n'
 'Frage: Wie läuft der Analyseprozess für Immobilienprojekte bei Engel & '
 'Völkers Digital Invest ab und

In [11]:
generated_result = invoke_llm_model(prompt, bedrock_client)
generated_result

{'inputTextTokenCount': 655,
 'results': [{'tokenCount': 218,
   'outputText': 'Analysekonzept;Wie viele externe Partner sind an dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest beteiligt?\nAnalysekonzept;Wer führt die Markt- und Standortanalyse bei dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest durch?\nAnalysekonzept;Wer prüft die rechtlichen Aspekte bei dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest?\nAnalysekonzept;Welche Firma analysiert den Projektentwickler bei dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest?\nAnalysekonzept;Wer übernimmt die technische Projektanalyse bei dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest?',
   'completionReason': 'FINISH'}]}

In [12]:
generated_questions = generated_result["results"][0]["outputText"].strip().split("\n")
for question in generated_questions:
    question_as_list = question.split(";")
    question_as_object = {
        "source_system": first_doc["source_system"],
        "category": question_as_list[0],
        "question": question_as_list[1],
        "document_id": first_doc["document_id"]
    }
    print(question_as_object)

{'source_system': 'evdi', 'section': 'Analysekonzept', 'question': 'Wie viele externe Partner sind an dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest beteiligt?', 'document_id': 'f2624f5125f9'}
{'source_system': 'evdi', 'section': 'Analysekonzept', 'question': 'Wer führt die Markt- und Standortanalyse bei dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest durch?', 'document_id': 'f2624f5125f9'}
{'source_system': 'evdi', 'section': 'Analysekonzept', 'question': 'Wer prüft die rechtlichen Aspekte bei dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest?', 'document_id': 'f2624f5125f9'}
{'source_system': 'evdi', 'section': 'Analysekonzept', 'question': 'Welche Firma analysiert den Projektentwickler bei dem Analyseprozess für Immobilienprojekte bei Engel & Völkers Digital Invest?', 'document_id': 'f2624f5125f9'}
{'source_system': 'evdi', 'section': 'Analysekonzept', 'question': 'Wer übernimmt die technische Proj

#### Generate ground truth questions for all items

In [13]:
def generate_questions_for_prompt(doc: Dict[str, Any], prompt: str, bedrock_client) -> List[Dict[str, Any]]:
    results = []
    generated_result = invoke_llm_model(prompt, bedrock_client)
    generated_questions = generated_result["results"][0]["outputText"].strip().split("\n")

    for question in generated_questions:
        question_as_list = question.split(";")
        question_as_object = {
            "source_system": doc["source_system"],
            "category": question_as_list[0].strip(),
            "question": question_as_list[1].strip(),
            "document_id": doc["document_id"]
        }

        results.append(question_as_object)

    return results

In [14]:
existing_df: DataFrame
if os.path.exists(ground_truth_path):
    existing_df = pd.read_csv(ground_truth_path, delimiter=";")
else:
    columns = ['source_system', 'category', 'question', 'document_id']
    existing_df = pd.DataFrame(columns=columns)

existing_df[:10]

,source_system,section,question,document_id
0,evdi,Analysekonzept,Was sind die wichtigsten Schritte des Analysep...,f2624f5125f9
1,evdi,Analysekonzept,Welche externen Partner sind an der Analyse vo...,f2624f5125f9
2,evdi,Analysekonzept,Wie garantiert Engel & Völkers Digital Invest ...,f2624f5125f9
3,evdi,Analysekonzept,Wer führt die technische Projektanalyse für Im...,f2624f5125f9
4,evdi,Analysekonzept,Wer überprüft die rechtlichen Aspekte von Immo...,f2624f5125f9


In [19]:
for idx, doc in tqdm(df.iterrows()):
    document_id_to_check = doc["document_id"]
    if not existing_df.empty and document_id_to_check in existing_df["document_id"].values:
        continue

    doc_as_dict = doc.to_dict()
    prompt = create_prompt_for_document(doc_as_dict)
    generated_questions = generate_questions_for_prompt(doc_as_dict, prompt, bedrock_client)

    documents_df = pd.DataFrame(generated_questions)
    existing_df = pd.concat([existing_df, documents_df], ignore_index=True)
    existing_df.to_csv(ground_truth_path, sep=";", index=False)

87it [04:56,  3.41s/it]


In [21]:
existing_df.head()

,source_system,section,question,document_id
0,evdi,Analysekonzept,Was sind die wichtigsten Schritte des Analysep...,f2624f5125f9
1,evdi,Analysekonzept,Welche externen Partner sind an der Analyse vo...,f2624f5125f9
2,evdi,Analysekonzept,Wie garantiert Engel & Völkers Digital Invest ...,f2624f5125f9
3,evdi,Analysekonzept,Wer führt die technische Projektanalyse für Im...,f2624f5125f9
4,evdi,Analysekonzept,Wer überprüft die rechtlichen Aspekte von Immo...,f2624f5125f9


In [20]:
existing_df.tail()

,source_system,section,question,document_id
441,evdi,Unser Leistungsnachweis,Welches Konzept hilft bei der Steigerung der R...,7e625586f6b2
442,evdi,Unser Leistungsnachweis,"Was ist der Name des Konzepts, das zur Steiger...",7e625586f6b2
443,evdi,Unser Leistungsnachweis,Was ist die Bezeichnung für das Reinvestment v...,7e625586f6b2
444,evdi,Unser Leistungsnachweis,"Was betont die Plattform, wenn es um das Reinv...",7e625586f6b2
445,evdi,Unser Leistungsnachweis,Was bietet die Plattform für die Reinvestition...,7e625586f6b2
